# Projet - ITK/VTK - Etude longitudinale de l’évolution d’une tumeur

In [1]:
import numpy as np
import itk
import matplotlib.pyplot as plt
print(itk.Version.GetITKVersion())

In [2]:
# get the output image
PixelType = itk.F

images_1 = itk.imread("./Data/case6_gre1.nrrd", PixelType)
images_2 = itk.imread("./Data/case6_gre2.nrrd", PixelType)

images_1.shape, images_2.shape

In [3]:
def print_5_slices(images, index_slice, dimension) :
    fig, axs = plt.subplots(1, 5, figsize=(18, 8))
    
    if dimension == 1 :
        if index_slice > images.shape[0] :
            raise ValueError("index_slice is higher than the number of slices")
        
        limit = index_slice
        i = 0
        while limit < images.shape[0] and i < 5 :
            limit += 1
            i += 1
        
        j = 0
        for i in range(index_slice, limit) :
            axs[j].imshow(images[i, :, :], cmap='gray')
            axs[j].axis('off')
            j += 1
    
    elif dimension >= 2 :
        if index_slice > images.shape[1] :
            raise ValueError("index_slice is higher than the number of slices")
        
        limit = index_slice
        i = 0
        while limit < images.shape[0] and i < 5 :
            limit += 1
            i += 1

        j = 0
        if dimension == 2 :
            for i in range(index_slice, limit) :
                axs[j].imshow(images[:, i, :], cmap='gray')
                axs[j].axis('off')
                j += 1
        else :
            for i in range(index_slice, limit) :
                axs[j].imshow(images[:, :, i], cmap='gray')
                axs[j].axis('off')
                j += 1
                
    plt.subplots_adjust(wspace=0.05)

print_5_slices(images_1, 100, 1)

In [4]:
print_5_slices(images_1, 100, 2)

In [5]:
print_5_slices(images_1, 150, 3)

## Etape 1 - Recalage

In [6]:
fig, axes = plt.subplots(3, 2, figsize=(8, 10))
axes[0, 0].imshow(images_1[100, :, :], cmap='gray')
axes[0, 1].imshow(images_2[100, :, :], cmap='gray')
axes[1, 0].imshow(images_1[:, 100, :], cmap='gray')
axes[1, 1].imshow(images_2[:, 100, :], cmap='gray')
axes[2, 0].imshow(images_1[:, :, 150], cmap='gray')
axes[2, 1].imshow(images_2[:, :, 150], cmap='gray')

On peut remarquer qu'il y a une différence marquée d'aligement latéral entre les deux images. Nous effectuerons donc un recalage par translation pour aligner nos images.

In [ ]:
dimension = 3
FixedImageType = type(images_1)
MovingImageType = type(images_2)

TransformType = itk.TranslationTransform[itk.D, dimension]
initialTransform = TransformType.New()

optimizer = itk.RegularStepGradientDescentOptimizerv4.New()

optimizer.SetLearningRate(1)
optimizer.SetMinimumStepLength(1e-6)
optimizer.SetNumberOfIterations(100)

metric = itk.MeanSquaresImageToImageMetricv4[FixedImageType, MovingImageType].New()
fixed_interpolation = itk.LinearInterpolateImageFunction[FixedImageType, itk.D].New()
metric.SetFixedInterpolator(fixed_interpolation)

registration = itk.ImageRegistrationMethodv4[FixedImageType, MovingImageType].New(FixedImage=images_1, MovingImage=images_2, Metric=metric,
                                                                                    Optimizer=optimizer, InitialTransform=initialTransform)

moving_initial_transform = TransformType.New()
initial_parameters = moving_initial_transform.GetParameters()
initial_parameters[0] = 0
initial_parameters[1] = 0
moving_initial_transform.SetParameters(initial_parameters)
registration.SetMovingInitialTransform(moving_initial_transform)

identity_transform = TransformType.New()
identity_transform.SetIdentity()
registration.SetFixedInitialTransform(identity_transform)

registration.SetNumberOfLevels(1)

registration.Update()

transform = registration.GetTransform()
final_parameters = transform.GetParameters()
translation_along_x = final_parameters.GetElement(0)
translation_along_y = final_parameters.GetElement(1)
translation_along_z = final_parameters.GetElement(2)

number_of_iterations = optimizer.GetCurrentIteration()

best_value = optimizer.GetValue()

print("Result = ")
print(" Translation X = " + str(translation_along_x))
print(" Translation Y = " + str(translation_along_y))
print(" Translation Z = " + str(translation_along_z))
print(" Iterations    = " + str(number_of_iterations))
print(" Metric value  = " + str(best_value))

In [ ]:
resampler = itk.ResampleImageFilter.New(Input=images_2, Transform=transform, UseReferenceImage=True,
                                        ReferenceImage=images_1)
resampler.SetDefaultPixelValue(0)

OutputImageType = itk.Image[PixelType, dimension]
images_2_resampled_casted = itk.CastImageFilter[FixedImageType, OutputImageType].New(resampler)

images_2_resampled = itk.array_from_image(images_2_resampled_casted)

In [ ]:
print_5_slices(images_2_resampled, 100, 1)

In [ ]:
print_5_slices(images_2_resampled, 100, 2)

In [ ]:
print_5_slices(images_2_resampled, 150, 3)

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(8, 10))
axes[0, 0].imshow(images_1[100, :, :], cmap='gray')
axes[0, 1].imshow(images_2_resampled[100, :, :], cmap='gray')
axes[1, 0].imshow(images_1[:, 100, :], cmap='gray')
axes[1, 1].imshow(images_2_resampled[:, 100, :], cmap='gray')
axes[2, 0].imshow(images_1[:, :, 150], cmap='gray')
axes[2, 1].imshow(images_2_resampled[:, :, 150], cmap='gray')

## Etape 2 - Ségmentation

400 - 775
position : 154 -177 85
index : 44 35 50

In [ ]:
dimension = 3
ImageType = type(images_1)

# Instantiate the filter
connected_threshold = itk.ConnectedThresholdImageFilter[ImageType, ImageType].New(images_1)

# Configure filter from the command line arguments
connected_threshold.SetReplaceValue(1)
connected_threshold.SetLower(0)
connected_threshold.SetUpper(500)

connected_threshold.SetSeed((44, 45, 50))

# Execute pipeline
connected_threshold.Update()

in_type = itk.output(connected_threshold)
output_type = itk.Image[itk.UC, dimension]
images_1_segmented_rescaled = itk.RescaleIntensityImageFilter[in_type, output_type].New(connected_threshold)

images_1_segmented = itk.array_from_image(images_1_segmented_rescaled)

fig, axes = plt.subplots(3, 2, figsize=(8, 10))
axes[0, 0].imshow(images_1[44, :, :], cmap='gray')
axes[0, 1].imshow(images_1_segmented[44, :, :], cmap='gray')
axes[1, 0].imshow(images_1[:, 35, :], cmap='gray')
axes[1, 1].imshow(images_1_segmented[:, 35, :], cmap='gray')
axes[2, 0].imshow(images_1[:, :, 50], cmap='gray')
axes[2, 1].imshow(images_1_segmented[:, :, 50], cmap='gray')